# [АиФ Доброе сердце](https://dobroe.aif.ru/)

 - [Ссылка на дашборд](https://datalens.yandex/4f3n1abvoiaqr)

**О проекте**: Благотворительный фонд АиФ Доброе сердце на данном этапе хочет автоматизировать свою аналитическую работу и задача аналитиков помочь ему в этом. Проект по работе с данными благотворительного фонда “Доброе сердце”. В датасетах представлены данные по действиям благотворителей за период с марта 2022 года по ноябрь 2023 года, а также данные о пожертвованиях благотворителей с 2021 года по ноябрь 2023 года. Данные хорошего качества, предобработка не требуется, при анализе, для которого используются оба датасета (действия, платежи) необходимо корректировать даты датасета с платежами. Также данные достаточно объемные, поэтому придется затронуть некоторые аспекты загрузки данных подобного размера (работа с ограниченным количеством строк при первичном анализе датасета, выбор столбцов для работы и загрузка датасета только с необходимыми столбцами).

**Цель проекта:** Фонд хочет лучше узнать своих благотворителей для более эффективной работы с ними. Для этого основной задачей будет проведения RFM-анализа, когортного анализа (retention, LTV, средний чек), а также расчет основных маркетинговых и продуктовых метрик. Также для достижения целей, поставленных фондом мы проанализируем эффективность каналов привлечения пользователей. По итогу должен получится рабочий скрипт для загрузки и подготовки данных для дальнейшей работы с ними + витрина данных, построенная по результатам работы скрипта. Таким образом будет пройден весь процесс ETL (extract - transform - load) и построена аналитическую панель.

## Техническое задание


### Шаг 1. Создание скрипта-загрузчика (облачное хранилище - база данных)

 - Подключение к яндекс диску (библиотека yadisk)
 - Собрать список файлов из папки АиФ Доброе сердце
 - Определить какие файлы уже были залиты в базу данных, а какие нет
 - Скачать новые файлы (при наличии)
 - Обработать новые файлы (пользовательские данные слить в один датасет, если файлов несколько)
 - Загрузить обработанные датасеты в буферные таблицы базы данных
 - Проверить загруженные данные
 - Загрузить данные в рабочие таблицы базы данных

### Шаг 2. Создание витрины данных

 - Подключиться к базе данных
 - Создать витрину данных (DataLens) на основании информации из БД
 - Витрина данных должна содержать следующую информацию (выполняется через SQL-запросы):
   - RFM-анализ
   - Когортный анализ: Retention Rate, LTV, AC
   - DAU, WAU, MAU, sticky factor.
   - Основные маркетинговые метрики
 
## Описание столбцов

Описание столбцов в датасете `public.payments` (данные о платежах пользователей (1 файл)):

- `OrderFirstActionDateTimeUtc` - Дата и время первого действия.
- `OrderTotalPrice` - Полная стоимость заказа (используется для расчётов).
- `OrderLineStatusIdsExternalId` - Статус линии (для анализа платежей используем значения: "paid", "notpaid", "fail" для анализа отказов и ошибок).
- `OrderCustomerIdsMindboxId` - Уникальный идентификатор пользователя (для слияния данных).
- `OrderFirstActionChannelName` - Название канала.

Описание столбцов в датасете `public.users_original` (данные о действиях пользователей (4 файла)):

- `CustomerActionDateTimeUtc` - Время и дата действия пользователя.
- `CustomerActionChannelName` - Название канала привлечения.
- `CustomerActionCustomerIdsMindboxId` - Уникальный идентификатор пользователя (для слияния датасетов).

Описание столбцов в датасете `public.users` (данные о действиях пользователей (4 файла)):

- `action_datetime` - Время и дата действия пользователя.
- `channel_name` - Название канала привлечения.
- `user_id` - Уникальный идентификатор пользователя (для слияния датасетов).

### Загрузка и предобработка данных

#### Импортируем библиотеки и модули

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
import datetime as dt
import yadisk  # Импортируем библиотеку для работы с API Яндекс.Диска
import sqlite3 as sl  # Импортируем модуль для работы с базой данных SQLite
import os  # Импортируем модуль для работы с файловой системой (создание директорий, работа с путями)
import sys  # Импортируем модуль sys для работы с системными параметрами и аргументами
import pickle  # Импортируем модуль для сериализации данных (сохранение и загрузка объектов в файлы)
from tqdm import tqdm  # Импортируем функцию tqdm для отображения прогресс-бара при выполнении циклов

#### Пишем скрипт-загрузчик

In [2]:
app_id = ''  # ID приложения для доступа к Яндекс.Диску
secret_id = ''  # Секретный ключ приложения
ya_token = ''  # Токен для авторизации

y = yadisk.YaDisk(app_id, secret_id, ya_token)  # Создаем объект для взаимодействия с API Яндекс.Диска

# Проверяем токен
if y.check_token():  # Проверяем, действителен ли токен
    print('Токен правильный')  # Сообщение о корректности токена

# Путь к папке с локальными файлами
load_path = 'F:/Датасеты/АиФ/'  # Указываем путь к локальной папке, куда будут скачиваться файлы
if not os.path.exists(load_path):  # Проверяем, существует ли папка
    os.mkdir(load_path)  # Если папка не существует, создаем её

# Файл для хранения информации о скачанных файлах
metadata_file = os.path.join(load_path, 'metadata.pkl')  # Путь к файлу с метаданными (информация о скачанных файлах)

# Загружаем метаданные о скачанных файлах, если они существуют
if os.path.exists(metadata_file):  # Проверяем, существует ли файл с метаданными
    with open(metadata_file, 'rb') as f:  # Открываем файл для чтения
        downloaded_files = pickle.load(f)  # Загружаем данные о скачанных файлах
else:
    downloaded_files = {}  # Если файл не существует, создаем пустой словарь

# Создаем список файлов на Яндекс Диске
print('Создание списка файлов')  # Сообщение о начале создания списка файлов
list_of_files = []  # Пустой список для хранения файлов

for el in tqdm(list(y.listdir('АиФ'))):  # Проходимся по всем файлам в папке "АиФ" на Яндекс.Диске
    if el['path'].endswith('.csv'):  # Если файл имеет расширение .csv
        file_path = el['path']  # Сохраняем путь к файлу
        file_modified = el['modified']  # Получаем дату последнего изменения файла
        list_of_files.append((file_path, file_modified))  # Добавляем путь и дату изменения в список файлов

# Проверяем файлы на наличие новых или изменённых
new_files = []  # Список для хранения новых или изменённых файлов
for file_path, file_modified in list_of_files:  # Проходимся по каждому файлу в списке
    # Если файла нет в списке скачанных или он был изменён, добавляем его в список для загрузки
    if (file_path not in downloaded_files) or (downloaded_files[file_path] != file_modified):  
        new_files.append(file_path)  # Добавляем файл в список для загрузки

if new_files:  # Если есть новые или обновлённые файлы
    print('Загрузка новых или обновленных файлов...')  # Сообщаем о начале загрузки
    for file in tqdm(new_files):  # Проходимся по каждому новому или изменённому файлу
        local_file_name = os.path.join(load_path, file.split('/')[-1])  # Формируем имя файла для сохранения
        y.download(file.split(':')[1], local_file_name)  # Скачиваем файл с Яндекс.Диска на локальный компьютер

        # Обновляем метаданные после успешного скачивания
        for el in list(y.listdir('АиФ')):  # Ищем файл на Яндекс.Диске
            if el['path'] == file:  # Если путь совпадает
                downloaded_files[file] = el['modified']  # Обновляем дату последнего изменения файла в метаданных

    # Сохраняем обновлённые метаданные
    with open(metadata_file, 'wb') as f:  # Открываем файл с метаданными для записи
        pickle.dump(downloaded_files, f)  # Сохраняем обновлённые данные о скачанных файлах
else:
    print('Новых или обновленных файлов нет.')  # Сообщаем, что новых или изменённых файлов нет

Токен правильный
Создание списка файлов


100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 5006.33it/s]

Новых или обновленных файлов нет.


Написан скрипт-загрузчик. Он проверяет, скачаны ли уже файлы. Если скачаны, то он их скачивает повторно только если они были изменены.

#### Обрабатываем новые файлы (пользовательские данные слить в один датасет, если файлов несколько)

In [3]:
columns_to_load = ['OrderFirstActionDateTimeUtc', 'OrderTotalPrice', 'OrderFirstActionChannelName',
                   'OrderLineStatusIdsExternalId', 'OrderCustomerIdsMindboxId']

payments = pd.read_csv('F:\Датасеты\АиФ\Заказы.csv', sep=';', usecols=columns_to_load)

payments.head()

,OrderFirstActionDateTimeUtc,OrderFirstActionChannelName,OrderTotalPrice,OrderLineStatusIdsExternalId,OrderCustomerIdsMindboxId
0,27.01.2022 0:00,Административный сайт Mindbox,500,Paid,6959
1,29.01.2022 0:00,Административный сайт Mindbox,200,Paid,7103
2,31.01.2022 0:00,Административный сайт Mindbox,300,Paid,7321
3,06.02.2022 0:00,Административный сайт Mindbox,300,Paid,7143
4,28.02.2022 0:00,Административный сайт Mindbox,50,Paid,7079


Создан датафрейм `payments` c донатами пользователей.

In [4]:
# Путь к папке с файлами
data_path = 'F:\\Датасеты\\АиФ\\'
columns_to_load = ['CustomerActionDateTimeUtc', 'CustomerActionChannelName', 'CustomerActionCustomerIdsMindboxId']

# Список файлов
files = [
    os.path.join(data_path, f'{i} of 4 - Dobroaif - 04.09.2024 - 41884 - 9b4daafd-72bb-4951-b6fd-0d09eaf37387.csv')
    for i in range(1, 5)
]

# Чтение и объединение файлов
users = pd.concat([pd.read_csv(file, sep=';', usecols=columns_to_load) for file in files], ignore_index=True)

users.head()

,CustomerActionDateTimeUtc,CustomerActionChannelName,CustomerActionCustomerIdsMindboxId
0,19.03.2022 21:30,Сайт,2734
1,14.10.2021 15:50,Административный сайт Mindbox,2734
2,18.03.2021 10:52,Административный сайт Mindbox,2734
3,23.11.2022 8:40,Сайт,2734
4,23.11.2022 8:40,utm_term Не указан,2734


Создан датафрейм `users` объединённый из 4-х датасетов, c действиями пользователей.

In [5]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96565 entries, 0 to 96564
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   OrderFirstActionDateTimeUtc   96565 non-null  object
 1   OrderFirstActionChannelName   96565 non-null  object
 2   OrderTotalPrice               96565 non-null  int64 
 3   OrderLineStatusIdsExternalId  96565 non-null  object
 4   OrderCustomerIdsMindboxId     96565 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 3.7+ MB


In [6]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3834751 entries, 0 to 3834750
Data columns (total 3 columns):
 #   Column                              Dtype 
---  ------                              ----- 
 0   CustomerActionDateTimeUtc           object
 1   CustomerActionChannelName           object
 2   CustomerActionCustomerIdsMindboxId  int64 
dtypes: int64(1), object(2)
memory usage: 87.8+ MB


#### Меням тип данных

In [7]:
# Указываем параметр dayfirst=True, чтобы интерпретировать даты в формате день/месяц/год
users['CustomerActionDateTimeUtc'] = pd.to_datetime(users['CustomerActionDateTimeUtc'], dayfirst=True)
payments['OrderFirstActionDateTimeUtc'] = pd.to_datetime(payments['OrderFirstActionDateTimeUtc'], dayfirst=True)

Изменены типы данных в колонках `CustomerActionDateTimeUtc` из датафрейма `users` и `OrderFirstActionDateTimeUtc` из датафрейма `payments` на datetime.

#### Загружаем обработанные датасеты в буферные таблицы базы данных

In [8]:
engine = sl.connect('aif_etl.db') # Создаем движок подключения к базе данных при помощи sqlite

#### Заливаем датасет в базу данных, где `users` - название таблицы в базе данных, `con (connector)` это наше соединение с базой данных `engine` и `if_exists` - определяет, что делать с данными, если они уже есть в базе данных, я выставил заменить.

In [9]:
users.to_sql('users', con=engine, if_exists='replace')
payments.to_sql('payments', con=engine, if_exists='replace')

96565

#### Проверяем корректность создания базы данных через pandas read sql. Указываем SQL запрос, который мы хотим выполнить и собственно также указываем наш коннектор к базе данных.

In [10]:
users = pd.read_sql('select * from users', con=engine)

In [11]:
payments = pd.read_sql('select * from payments', con=engine)

#### Проверяем данные

In [12]:
users.head()

,index,CustomerActionDateTimeUtc,CustomerActionChannelName,CustomerActionCustomerIdsMindboxId
0,0,2022-03-19 21:30:00,Сайт,2734
1,1,2021-10-14 15:50:00,Административный сайт Mindbox,2734
2,2,2021-03-18 10:52:00,Административный сайт Mindbox,2734
3,3,2022-11-23 08:40:00,Сайт,2734
4,4,2022-11-23 08:40:00,utm_term Не указан,2734


In [13]:
payments.head()

,index,OrderFirstActionDateTimeUtc,OrderFirstActionChannelName,OrderTotalPrice,OrderLineStatusIdsExternalId,OrderCustomerIdsMindboxId
0,0,2022-01-27 00:00:00,Административный сайт Mindbox,500,Paid,6959
1,1,2022-01-29 00:00:00,Административный сайт Mindbox,200,Paid,7103
2,2,2022-01-31 00:00:00,Административный сайт Mindbox,300,Paid,7321
3,3,2022-02-06 00:00:00,Административный сайт Mindbox,300,Paid,7143
4,4,2022-02-28 00:00:00,Административный сайт Mindbox,50,Paid,7079


#### Проверяем наличие явных дубликатов и обработать их при необходимости

In [14]:
print(payments.duplicated().sum())
print(users.duplicated().sum())

0
0


Явных дубликатов нет

#### Проверяем на наличие пропусков

In [15]:
def check_missing_values(df, df_name):
    """
    Функция для подсчета и отображения пропущенных значений в столбцах датафрейма.
    """
    # Подсчет пропусков по колонкам
    report = df.isna().sum()

    # Фильтрация только колонок с пропусками
    report = report[report > 0].to_frame(name='missing_values')

    # Проверка наличия пропусков и создание отчета
    if not report.empty:
        # Добавление столбца с процентами пропусков
        report['% of total'] = (report['missing_values'] / df.shape[0]).round(3) * 100
        report = report.sort_values(by='missing_values', ascending=False)
        display(report)
    else:
        print(f"Пропусков в таблице {df_name} нет.")

check_missing_values(payments, 'payments')
check_missing_values(users, 'users')

Пропусков в таблице payments нет.
Пропусков в таблице users нет.


### Исследовательский анализ данных (EDA)

In [16]:
# Словарь с описаниями столбцов
column_descriptions = {
    'OrderFirstActionDateTimeUtc': 'Дата и время первого действия (UTC)',
    'OrderTotalPrice': 'Полная стоимость заказа',
    'OrderLineStatusIdsExternalId': 'Внешний идентификатор статуса линии',
    'OrderCustomerIdsMindboxId': 'Уникальный идентификатор клиента'
}

# Основной код для вывода уникальных значений и описания столбцов
for col, description in column_descriptions.items():
    unique_values = payments[col].unique()  # Получаем уникальные значения
    unique_count = len(unique_values)  # Количество уникальных значений

    print(f"Столбец: {col} ({description})")  # Выводим столбец и его описание
    print(f"Уникальных значений: {unique_count}")
    
    # Выводим максимум 5 уникальных значений
    print(unique_values[:5])  
    
    # Если уникальных значений больше 5, показываем, что есть еще
    if unique_count > 5:
        print('...')  # Указываем, что есть больше уникальных значений
    
    print('-' * 50)  # Разделитель для наглядности

Столбец: OrderFirstActionDateTimeUtc (Дата и время первого действия (UTC))
Уникальных значений: 71078
['2022-01-27 00:00:00' '2022-01-29 00:00:00' '2022-01-31 00:00:00'
 '2022-02-06 00:00:00' '2022-02-28 00:00:00']
...
--------------------------------------------------
Столбец: OrderTotalPrice (Полная стоимость заказа)
Уникальных значений: 812
[ 500  200  300   50 2000]
...
--------------------------------------------------
Столбец: OrderLineStatusIdsExternalId (Внешний идентификатор статуса линии)
Уникальных значений: 3
['Paid' 'fail' 'notpaid']
--------------------------------------------------
Столбец: OrderCustomerIdsMindboxId (Уникальный идентификатор клиента)
Уникальных значений: 26206
[6959 7103 7321 7143 7079]
...
--------------------------------------------------


In [17]:
# Словарь с описаниями столбцов для датасета users
column_descriptions_users = {
    'CustomerActionDateTimeUtc': 'Время и дата действия пользователя (UTC)',
    'CustomerActionChannelName': 'Название канала',
    'CustomerActionCustomerIdsMindboxId': 'Идентификатор пользователя'
}

# Основной код для вывода уникальных значений и описания столбцов для users
for col, description in column_descriptions_users.items():
    unique_values = users[col].unique()  # Получаем уникальные значения
    unique_count = len(unique_values)  # Количество уникальных значений

    print(f"Столбец: {col} ({description})")  # Выводим столбец и его описание
    print(f"Уникальных значений: {unique_count}")
    
    # Выводим максимум 5 уникальных значений
    print(unique_values[:5])  
    
    # Если уникальных значений больше 5, показываем, что есть еще
    if unique_count > 5:
        print('...')  # Указываем, что есть больше уникальных значений
    
    print('-' * 50)  # Разделитель для наглядности

Столбец: CustomerActionDateTimeUtc (Время и дата действия пользователя (UTC))
Уникальных значений: 388121
['2022-03-19 21:30:00' '2021-10-14 15:50:00' '2021-03-18 10:52:00'
 '2022-11-23 08:40:00' '2022-11-23 08:41:00']
...
--------------------------------------------------
Столбец: CustomerActionChannelName (Название канала)
Уникальных значений: 342
['Сайт' 'Административный сайт Mindbox' 'utm_term Не указан' 'Email'
 'Прямой переход']
...
--------------------------------------------------
Столбец: CustomerActionCustomerIdsMindboxId (Идентификатор пользователя)
Уникальных значений: 28633
[2734 2847 2861 3361 4982]
...
--------------------------------------------------


In [18]:
users['CustomerActionChannelName'].value_counts().head(20)

CustomerActionChannelName
Email                            2781285
Сайт                              811895
Административный сайт Mindbox      85143
utm_term Не указан                 64523
Прямой переход                     43904
vk.com                              8644
WebPush                             7729
yandex.ru                           6180
google.com                          3842
instagram.com                       2819
yoomoney.ru                         2039
roditeli                            1978
razovie                              990
report                               735
lina                                 648
m.news.yandex.ru                     536
ida                                  469
ok.ru                                418
ru.yandex.weatherplugin              388
m.video.yandex.ru                    358
Name: count, dtype: int64

### Проверка гипотез

**Гипотеза 1: Сезон и частота пожертвований**

- **Нулевая гипотеза (H₀):** Частота пожертвований не зависит от месяца.  
- **Альтернативная гипотеза (H₁):** Частота пожертвований зависит от месяца.  
- **Критический уровень значимости (α):** 0.05.  
- **Применяемый тест:** χ²-тест для равномерного распределения.  

In [19]:
# Преобразуем дату первого действия в дату и выделяем месяц
payments['OrderMonth'] = pd.to_datetime(payments['OrderFirstActionDateTimeUtc']).dt.month

# Подсчитаем частоту пожертвований по месяцам
monthly_payments = payments['OrderMonth'].value_counts().sort_index()

# Выводим пропорции пожертвований по месяцам
print("Частота заказов по месяцам:")
print(monthly_payments)

# Ожидаемое равномерное распределение (все месяцы одинаково вероятны)
# Рассчитаем ожидаемую частоту как общее количество пожертвований, делённое на число месяцев
expected_frequencies = [len(payments) / 12] * 12

# Выполняем χ²-тест
chi2, p_val = stats.chisquare(f_obs=monthly_payments, f_exp=expected_frequencies)

# Выводим результаты теста
print(f"Результаты χ²-теста:")
print(f"χ² = {chi2:.2f}, p-value = {p_val:.2f}")

# Интерпретация результатов
alpha = 0.05  # Уровень значимости
if p_val < alpha:
    print("Отклоняем нулевую гипотезу: частота заказов зависит от месяца.")
else:
    print("Не удалось отклонить нулевую гипотезу: частота заказов не зависит от месяца.")

Частота заказов по месяцам:
OrderMonth
1      6238
2      7476
3      7616
4      8379
5      8111
6      9371
7     10216
8     13166
9      7466
10     5423
11     5070
12     8033
Name: count, dtype: int64
Результаты χ²-теста:
χ² = 6542.24, p-value = 0.00
Отклоняем нулевую гипотезу: частота заказов зависит от месяца.


Результаты анализа показывают, что частота заказов действительно зависит от месяца, поскольку p-value меньше критического уровня значимости (alpha = 0.05). Это указывает на то, что распределение заказов по месяцам неравномерное.

**Гипотеза 2: Канал привлечения влияет на долю повторных заказов**

- **Нулевая гипотеза (H₀):** Доля повторных заказов не зависит от канала привлечения.  
- **Альтернативная гипотеза (H₁):** Доля повторных заказов зависит от канала привлечения.  
- **Критический уровень значимости (α):** 0.05.  
- **Применяемый тест:** χ²-тест для проверки связи между двумя категориальными переменными.

In [20]:
# Определим, у каких пользователей есть повторные заказы
repeated_orders = payments.groupby('OrderCustomerIdsMindboxId').size() > 1

# Добавим флаг повторных заказов в таблицу payments
payments['IsRepeated'] = payments['OrderCustomerIdsMindboxId'].isin(repeated_orders[repeated_orders].index)

# Подсчитаем количество повторных и неповторных заказов для каждого канала привлечения
channel_repeats = payments.groupby(['OrderFirstActionChannelName', 'IsRepeated']).size().unstack(fill_value=0)

# Вывод пропорций повторных и неповторных заказов по каналам
print("Пропорции повторных заказов по каналам:")
print(channel_repeats)

# Выполняем χ²-тест
chi2, p_val, dof, expected = chi2_contingency(channel_repeats)

# Выводим результаты теста
print("\nРезультаты χ²-теста:")
print(f"χ² = {chi2:.2f}, p-value = {p_val:.2f}")

# Интерпретация результатов
alpha = 0.05  # Уровень значимости
if p_val < alpha:
    print("Отклоняем нулевую гипотезу: доля повторных заказов зависит от канала привлечения.")
else:
    print("Не удалось отклонить нулевую гипотезу: доля повторных заказов не зависит от канала привлечения.")

Пропорции повторных заказов по каналам:
IsRepeated                     False  True 
OrderFirstActionChannelName                
Административный сайт Mindbox   2939  11353
Сайт                           12128  70145

Результаты χ²-теста:
χ² = 313.07, p-value = 0.00
Отклоняем нулевую гипотезу: доля повторных заказов зависит от канала привлечения.


Результаты анализа показали, что доля повторных заказов статистически значимо зависит от канала привлечения (χ² = 313.07, p-value = 0.00). Наибольшая доля повторных заказов наблюдается среди пользователей, привлеченных через сайт.

### Итоги

 - Написан скрипт-загрузчик. Он проверяет, скачаны ли уже файлы. Если скачаны, то он их скачивает повторно только если они были изменены.
 
 - Создан датафрейм `payments` c донатами пользователей.

 - Создан датафрейм `users` объединённый из 4-х датасетов, c действиями пользователей.

 - Изменены типы данных в колонках `CustomerActionDateTimeUtc` из датафрейма `users` и `OrderFirstActionDateTimeUtc` из датафрейма `payments` на datetime.
 
 - Явных дубликатов и пропусков в таблицах нет

 - Создана БД с загруженными таблицами
 
 - Проверены 2 гипотезы. В следствии которых было обнаружено что:
   - Частота заказов зависит от месяца.
   - Доля повторных заказов зависит от канала привлечения.

 - Создан дашборд